In [6]:
import torch
import pickle
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt
from torch_geometric.loader import DataLoader
from torch_geometric.data import Data

ModuleNotFoundError: No module named 'torch'

In [7]:
!dir

 Volume in drive C is OS
 Volume Serial Number is AEC9-A454

 Directory of C:\uni\ml4gd\periodic-graph-traffic-forecasting\src\notebooks

26/06/2023  22:31    <DIR>          .
23/06/2023  15:09    <DIR>          ..
23/06/2023  15:09    <DIR>          .ipynb_checkpoints
24/06/2023  13:41           124,998 basic_GNN.ipynb
26/06/2023  22:31            24,238 data_loading.ipynb
               2 File(s)        149,236 bytes
               3 Dir(s)  703,003,385,856 bytes free


In [8]:
%cd ../..

C:\uni\ml4gd\periodic-graph-traffic-forecasting


In [9]:
data = pd.read_csv('data/metr-la.csv')

In [10]:
data

,Unnamed: 0,773869,767541,767542,717447,717446,717445,773062,767620,737529,...,772167,769372,774204,769806,717590,717592,717595,772168,718141,769373
0,2012-03-01 00:00:00,64.375000,67.625000,67.125000,61.500000,66.875000,68.750000,65.125000,67.125000,59.625000,...,45.625000,65.500000,64.500000,66.428571,66.875000,59.375000,69.000000,59.250000,69.000000,61.875000
1,2012-03-01 00:05:00,62.666667,68.555556,65.444444,62.444444,64.444444,68.111111,65.000000,65.000000,57.444444,...,50.666667,69.875000,66.666667,58.555556,62.000000,61.111111,64.444444,55.888889,68.444444,62.875000
2,2012-03-01 00:10:00,64.000000,63.750000,60.000000,59.000000,66.500000,66.250000,64.500000,64.250000,63.875000,...,44.125000,69.000000,56.500000,59.250000,68.125000,62.500000,65.625000,61.375000,69.857143,62.000000
3,2012-03-01 00:15:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2012-03-01 00:20:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34267,2012-06-27 23:35:00,65.000000,65.888889,68.555556,61.666667,0.000000,54.555556,62.444444,63.333333,59.222222,...,52.888889,69.000000,65.111111,55.666667,66.333333,62.444444,66.777778,64.888889,69.666667,62.333333
34268,2012-06-27 23:40:00,61.375000,65.625000,66.500000,62.750000,0.000000,50.500000,62.000000,67.000000,65.250000,...,54.000000,69.250000,60.125000,60.500000,67.250000,59.375000,66.000000,61.250000,69.000000,62.000000
34269,2012-06-27 23:45:00,67.000000,59.666667,69.555556,61.000000,0.000000,44.777778,64.222222,63.777778,59.777778,...,51.333333,67.888889,64.333333,57.000000,66.000000,62.666667,68.666667,63.333333,67.444444,61.222222
34270,2012-06-27 23:50:00,66.750000,62.250000,66.000000,59.625000,0.000000,53.000000,64.285714,64.125000,60.875000,...,51.125000,69.375000,61.625000,60.500000,65.625000,66.375000,69.500000,63.000000,67.875000,63.500000


In [11]:
ids = np.loadtxt('data/graph_sensor_ids_small.csv', delimiter=",", dtype=int)
str_ids = [str(x) for x in ids] # for pandas

In [12]:
import pickle

with open('data/adj_mx.pkl', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    p = u.load()

id_map = p[1]
adj = p[2]

In [13]:
adj_ids = [id_map[str(i)] for i in ids]

In [14]:
reduced_adj = adj[adj_ids].T[adj_ids].T

In [15]:
from scipy.sparse.csgraph import laplacian, csgraph_from_dense

In [18]:
edge_index = []
features = []

for i, id_a in enumerate(adj_ids):
    for j, id_b in enumerate(adj_ids):
        if i == j:
            continue
        
        if reduced_adj[i, j] > 0:
            edge_index.append([i, j])
            features.append(reduced_adj[i, j])
            
edge_index = np.array(edge_index).T
features = np.array(features).reshape(-1, 1)

In [19]:
edge_index.T

array([[ 0,  1],
       [ 0,  2],
       [ 0,  7],
       [ 0, 17],
       [ 0, 23],
       [ 0, 43],
       [ 0, 46],
       [ 1,  0],
       [ 1,  7],
       [ 1, 10],
       [ 1, 17],
       [ 1, 31],
       [ 1, 33],
       [ 1, 43],
       [ 1, 46],
       [ 2,  3],
       [ 2,  6],
       [ 2, 18],
       [ 2, 19],
       [ 2, 23],
       [ 2, 24],
       [ 2, 26],
       [ 2, 34],
       [ 2, 45],
       [ 2, 48],
       [ 3, 14],
       [ 3, 22],
       [ 3, 24],
       [ 3, 34],
       [ 3, 41],
       [ 3, 45],
       [ 4, 12],
       [ 4, 15],
       [ 4, 16],
       [ 4, 20],
       [ 4, 29],
       [ 4, 32],
       [ 4, 39],
       [ 4, 40],
       [ 4, 41],
       [ 4, 42],
       [ 5, 30],
       [ 5, 31],
       [ 5, 33],
       [ 5, 36],
       [ 5, 37],
       [ 6,  0],
       [ 6,  1],
       [ 6,  2],
       [ 6,  7],
       [ 6, 17],
       [ 6, 18],
       [ 6, 23],
       [ 6, 46],
       [ 7,  2],
       [ 7,  3],
       [ 7,  6],
       [ 7, 18],
       [ 7, 19

In [28]:
def create_spatiotemporal_data(adjacency, horizon, days_back=np.array([1, 1, 0, 0, 0, 0, 0, 1]), time_weight=0.1):
    n_nodes = len(adjacency)
    active_replicas = horizon * np.count_nonzero(days_back)

    node_offsets = np.arange(active_replicas) * n_nodes

    
    edge_index_orig = []
    features_orig = []

    for i in range(n_nodes):
        for j in range(n_nodes):
            if i == j:
                continue

            if adjacency[i, j] > 0:
                edge_index_orig.append([i, j])
                features_orig.append(adjacency[i, j])

    edge_index_orig = np.array(edge_index_orig)
    features_orig = list(features_orig)

    edge_index = []
    features = features_orig * active_replicas

    for offset in node_offsets:
        edge_index = [*edge_index, *(edge_index_orig + offset)]

    for i in range(n_nodes):
        for offset in node_offsets[:-1]:
            from_node = offset + i + n_nodes
            to_node = offset + i
            edge_index.append([from_node, to_node])
            edge_index.append([to_node, from_node])
            features.append(time_weight)
            features.append(time_weight)
            
    for i in range(n_nodes):
        for offset in node_offsets[int(horizon/2)-1::horizon][1:]:
            from_node = offset + i + n_nodes
            edge_index.append([from_node, i])
            edge_index.append([i, from_node])
            features.append(time_weight)
            features.append(time_weight)

    edge_index = np.array(edge_index)
    features = np.array(features).reshape(-1, 1)

    day_timesteps = 24 * 60 / 5
    time_offsets = []

    for i, day in enumerate(days_back):
        if i == 0:
            time_offsets.append(np.arange(horizon))

        elif day == 0:
            continue

        else:
            time_offsets.append(np.arange(horizon) - int(horizon/2) + i * day_timesteps)

    time_offsets = np.array(time_offsets, dtype=int).reshape(-1)

    print(len(edge_index), len(features))
    return edge_index, features, time_offsets


edge_index, features, time_offsets = create_spatiotemporal_data(np.array([[1, 1], [0, 1]]), 6)
edge_index, time_offsets

94 94


(array([[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15],
        [16, 17],
        [18, 19],
        [20, 21],
        [22, 23],
        [24, 25],
        [26, 27],
        [28, 29],
        [30, 31],
        [32, 33],
        [34, 35],
        [ 2,  0],
        [ 0,  2],
        [ 4,  2],
        [ 2,  4],
        [ 6,  4],
        [ 4,  6],
        [ 8,  6],
        [ 6,  8],
        [10,  8],
        [ 8, 10],
        [12, 10],
        [10, 12],
        [14, 12],
        [12, 14],
        [16, 14],
        [14, 16],
        [18, 16],
        [16, 18],
        [20, 18],
        [18, 20],
        [22, 20],
        [20, 22],
        [24, 22],
        [22, 24],
        [26, 24],
        [24, 26],
        [28, 26],
        [26, 28],
        [30, 28],
        [28, 30],
        [32, 30],
        [30, 32],
        [34, 32],
        [32, 34],
        [ 3,  1],
        [ 1,  3],
        [ 5,  3],
        [ 

In [70]:
horizon = 12
train_test_val_split = [0.8, 0.1, 0.1]
n = 10000 # make sure this number is smaller than len(data) - 2 * horizon

In [71]:
train = []
test = []
val = []

np_data = data[str_ids].to_numpy()

for i in range(np.max(time_offsets)+1, n):
    data_offsetted_x = []
    data_offsetted_y = []
    
    for offset in -time_offsets:
        new_x = []
        time_encoding = encode_time(data[data.columns[0]][i + offset])
        for measurement in np_data[i + offset].T:
            new_x.append([*time_encoding, measurement])
        data_offsetted_x.append(new_x)
        data_offsetted_y.append(np_data[i + offset + 1 : i + offset + horizon + 1].T)
#         print(offset)
#         print(data_offsetted_y[-1].shape)
#     print(data_offsetted_x)
        
    data_offsetted_x = np.array(data_offsetted_x).reshape(-1, 5)
    data_offsetted_y = np.array(data_offsetted_y).reshape(-1, horizon)
    
    pyg_data = Data(x=torch.Tensor(data_offsetted_x),
            y=torch.Tensor(data_offsetted_y),
            edge_index=torch.Tensor(edge_index),
            edge_attr=torch.Tensor(features))

# for i in range(n):
#     pyg_data = Data(x=torch.Tensor(np_data[i:i+horizon].T),
#                 y=torch.Tensor(np_data[i+horizon:i+2*horizon].T),
#                 edge_index=torch.Tensor(edge_index),
#                 edge_attr=torch.Tensor(features))
    
    if (i / n < train_test_val_split[0]):
        train.append(pyg_data)
    elif (i / n < train_test_val_split[0] + train_test_val_split[1]):
        test.append(pyg_data)
    else:
        val.append(pyg_data)
    print(f"Sample {i+1} / {n}", end='\r')

In [77]:
train[800].x

tensor([[-9.7493e-01, -2.2252e-01, -1.0000e+00, -1.8370e-16,  6.2625e+01],
        [-9.7493e-01, -2.2252e-01, -1.0000e+00, -1.8370e-16,  6.7625e+01],
        [-9.7493e-01, -2.2252e-01, -1.0000e+00, -1.8370e-16,  6.2625e+01],
        ...,
        [-9.7493e-01, -2.2252e-01, -1.0000e+00, -1.8370e-16,  6.3875e+01],
        [-9.7493e-01, -2.2252e-01, -1.0000e+00, -1.8370e-16,  6.6000e+01],
        [-9.7493e-01, -2.2252e-01, -1.0000e+00, -1.8370e-16,  6.3250e+01]])

In [39]:
len(train), len(test), len(val)

(5978, 1000, 1000)

In [40]:
with open('data/Training_dataset_LA_small.p', 'wb') as handle:
    pickle.dump(train, handle)
with open('data/Test_dataset_LA_small.p', 'wb') as handle:
    pickle.dump(test, handle)
with open('data/Validation_dataset_LA_small.p', 'wb') as handle:
    pickle.dump(val, handle)

In [209]:
edge_index

array([[ 0,  1],
       [ 0,  2],
       [ 0,  7],
       [ 0, 17],
       [ 0, 23],
       [ 0, 43],
       [ 0, 46],
       [ 1,  0],
       [ 1,  7],
       [ 1, 10],
       [ 1, 17],
       [ 1, 31],
       [ 1, 33],
       [ 1, 43],
       [ 1, 46],
       [ 2,  3],
       [ 2,  6],
       [ 2, 18],
       [ 2, 19],
       [ 2, 23],
       [ 2, 24],
       [ 2, 26],
       [ 2, 34],
       [ 2, 45],
       [ 2, 48],
       [ 3, 14],
       [ 3, 22],
       [ 3, 24],
       [ 3, 34],
       [ 3, 41],
       [ 3, 45],
       [ 4, 12],
       [ 4, 15],
       [ 4, 16],
       [ 4, 20],
       [ 4, 29],
       [ 4, 32],
       [ 4, 39],
       [ 4, 40],
       [ 4, 41],
       [ 4, 42],
       [ 5, 30],
       [ 5, 31],
       [ 5, 33],
       [ 5, 36],
       [ 5, 37],
       [ 6,  0],
       [ 6,  1],
       [ 6,  2],
       [ 6,  7],
       [ 6, 17],
       [ 6, 18],
       [ 6, 23],
       [ 6, 46],
       [ 7,  2],
       [ 7,  3],
       [ 7,  6],
       [ 7, 18],
       [ 7, 19

In [43]:
time_offsets[:-1:horizon]

array([   0, 2010])

In [45]:
np_data

array([[67.625     , 67.125     , 67.125     , ..., 63.875     ,
        66.875     , 59.375     ],
       [68.55555556, 65.44444444, 65.        , ..., 63.88888889,
        62.        , 61.11111111],
       [63.75      , 60.        , 64.25      , ..., 67.25      ,
        68.125     , 62.5       ],
       ...,
       [59.66666667, 69.55555556, 63.77777778, ..., 66.88888889,
        66.        , 62.66666667],
       [62.25      , 66.        , 64.125     , ..., 64.625     ,
        65.625     , 66.375     ],
       [66.88888889, 66.77777778, 65.11111111, ..., 65.        ,
        64.22222222, 66.44444444]])

In [42]:
from datetime import datetime

In [61]:
def encode_time(timestamp):
    week_day = datetime.fromisoformat(timestamp).weekday()
    week_day_sin = np.sin(2 * np.pi * week_day / 7.0)
    week_day_cos = np.cos(2 * np.pi * week_day / 7.0)

    hour = datetime.fromisoformat(timestamp).hour
    hour_sin = np.sin(2 * np.pi * hour / 24.0)
    hour_cos = np.cos(2 * np.pi * hour / 24.0)

    return week_day_sin, week_day_cos, hour_sin, hour_cos

In [45]:
[encode_time(timestamp) for timestamp in data[data.columns[0]][time_offsets]]

[(0.43388373911755823,
  -0.900968867902419,
  1.0,
  6.123233995736766e-17,
  0.0,
  1.0),
 (0.43388373911755823,
  -0.900968867902419,
  1.0,
  6.123233995736766e-17,
  0.0,
  1.0),
 (0.43388373911755823,
  -0.900968867902419,
  1.0,
  6.123233995736766e-17,
  0.0,
  1.0),
 (0.43388373911755823,
  -0.900968867902419,
  1.0,
  6.123233995736766e-17,
  0.0,
  1.0),
 (0.43388373911755823,
  -0.900968867902419,
  1.0,
  6.123233995736766e-17,
  0.0,
  1.0),
 (0.43388373911755823,
  -0.900968867902419,
  1.0,
  6.123233995736766e-17,
  0.0,
  1.0),
 (0.43388373911755823,
  -0.900968867902419,
  1.0,
  6.123233995736766e-17,
  -0.25881904510252157,
  0.9659258262890681),
 (0.43388373911755823,
  -0.900968867902419,
  1.0,
  6.123233995736766e-17,
  -0.25881904510252157,
  0.9659258262890681),
 (0.43388373911755823,
  -0.900968867902419,
  1.0,
  6.123233995736766e-17,
  -0.25881904510252157,
  0.9659258262890681),
 (-0.433883739117558,
  -0.9009688679024191,
  1.0,
  6.123233995736766e-17,